In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uniprot/goa_cow.gaf
/kaggle/input/uniprot/uniprot_sprot.fasta
/kaggle/input/cafa-6-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-6-protein-function-prediction/IA.tsv
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset-taxon-list.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo


In [37]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("seddiktrk/protein-go-annotations")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/protein-go-annotations


In [2]:
!pip install biopython --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 67.5 MB/s eta 0:00:00:00:01


In [31]:
from pathlib import Path
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict
import subprocess
from subprocess import Popen, PIPE

In [51]:
def create_df_tsv(tsv_file):
    df = pd.read_csv(tsv_file, sep = "\t")
    return df

In [14]:
def create_df(data_root, seq_file, has_metadata):
    # create dataframe with 2 columns: EntryID + Seq
    root = Path(data_root)

    ids = []
    seqs = []

    with open(root / seq_file) as handle:
        for record in SeqIO.parse(handle, "fasta"):
            if has_metadata:
                id_only = record.id.split('|')[1]
                ids.append(id_only)
            else:
                ids.append(record.id)
            seqs.append(str(record.seq))
    final_df = pd.DataFrame({'EntryID': ids, 'Seq': seqs})
    return final_df

In [52]:
train_cafa6_df = create_df(
    data_root="/kaggle/input/cafa-6-protein-function-prediction",
    seq_file="Train/train_sequences.fasta",
    has_metadata=True
)

train_cafa5_df = create_df(
    data_root="/kaggle/input/cafa5-additional-data",
    seq_file="train_sequences.fasta",
    has_metadata=False
)

train_uniprot_df = create_df_tsv("/kaggle/input/final-uniprot/uni_seq.tsv")

test_df = create_df(
    data_root="/kaggle/input/cafa-6-protein-function-prediction",
    seq_file="Test/testsuperset.fasta",
    has_metadata=False
)

In [54]:
train_cafa5_df.head(5)

,EntryID,Seq
0,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...
1,O73864,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...
2,O95231,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...
3,A0A0B4J1F4,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
4,P54366,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...


In [55]:
test_df.head(5)

,EntryID,Seq
0,A0A0C5B5G6,MRWQEMGYIFYPRKLR
1,A0A1B0GTW7,MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...
2,A0JNW5,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...
3,A0JP26,MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...
4,A0PK11,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...


In [21]:
def create_file_from_df(df, name):
    records = []
    for _, row in df.iterrows():
        rec = SeqRecord(
            Seq(row['Seq']),
            id=row['EntryID'],
            description=""
        )
        records.append(rec)
    
    fasta_path = name
    SeqIO.write(records, fasta_path, "fasta")

In [56]:
create_file_from_df(train_cafa6_df, "train_cafa6.fasta")
create_file_from_df(train_cafa5_df, "train_cafa5.fasta")
create_file_from_df(train_uniprot_df, "train_uniprot.fasta")
create_file_from_df(test_df, "test.fasta")

In [24]:
def fix_duplicate(fasta_file, fasta_unique):
    # check if the file contains duplicate entries
    
    records = list(SeqIO.parse(fasta_file, "fasta"))

    # Lọc trùng: dùng (id, seq) làm key
    seen = set()
    unique_records = []
    for r in records:
        key = (r.id, str(r.seq))
        if key not in seen:
            seen.add(key)
            unique_records.append(r)
    if len(records) == len (unique_records):
        print("no duplicates found.")
    else:
        SeqIO.write(unique_records, fasta_unique, "fasta")
        print(f"{len(records)} -> {len(unique_records)} records unique")

In [57]:
fix_duplicate("/kaggle/working/train_cafa5.fasta", "train_cafa5_unique.fasta")
fix_duplicate("/kaggle/working/train_cafa6.fasta", "train_cafa6_unique.fasta")
fix_duplicate("/kaggle/working/train_uniprot.fasta", "train_uniprot_unique.fasta")

no duplicates found.
no duplicates found.
no duplicates found.


In [26]:
!wget http://github.com/bbuchfink/diamond/releases/download/v2.1.6/diamond-linux64.tar.gz
!tar xzf diamond-linux64.tar.gz
!rm diamond-linux64.tar.gz

URL transformed to HTTPS due to an HSTS policy
--2025-11-28 13:54:53--  https://github.com/bbuchfink/diamond/releases/download/v2.1.6/diamond-linux64.tar.gz
Resolving github.com (github.com)... 4.237.22.38
Connecting to github.com (github.com)|4.237.22.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/31987083/6b15e096-093f-4e8e-b9ff-5e7bb45db282?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-28T14%3A50%3A17Z&rscd=attachment%3B+filename%3Ddiamond-linux64.tar.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-28T13%3A49%3A33Z&ske=2025-11-28T14%3A50%3A17Z&sks=b&skv=2018-11-09&sig=Z3k%2Fv31pjvZo9YZFTdaj83gX2tLKteSIPjm58dkZGtw%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDMzOTg5NCwibmJmIjox

In [15]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

tsv_file = "train_uniprot.tsv"
fasta_file = "train_uniprot_db.fasta"

records = []

with open(tsv_file) as f:
    next(f)  # bỏ header
    for line in f:
        protein_id, sequence, _ = line.strip().split("\t")
        record = SeqRecord(Seq(sequence), id=protein_id, description="")
        records.append(record)

with open(fasta_file, "w") as f:
    SeqIO.write(records, f, "fasta")


In [28]:
def create_database(db_name, data_path):
    p = Popen(['./diamond', 'makedb', 
           '--in', data_path,
            '-d', db_name], stdin=PIPE, stdout=PIPE)
    stdout, stderr = p.communicate()

In [30]:
def matches(db_name, query_path, outfile_name, k):
    #outfile_name has to be .fasta file!
    p = Popen(['./diamond', 'blastp', '-d', db_name,
           '-q', query_path,
            '-o', outfile_name, '--max-target-seqs', str(k), '--quiet'], stdin=PIPE, stdout=PIPE)
    stdout, stderr = p.communicate()

In [58]:
create_database("train_cafa5_db", "/kaggle/working/train_cafa5.fasta")
create_database("train_cafa6_db", "/kaggle/working/train_cafa6.fasta")
create_database("train_uniprot_db", "/kaggle/working/train_uniprot.fasta")

diamond v2.1.6.160 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 4
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: /kaggle/working/train_cafa5.fasta
Opening the database file...  [0.003s]
Loading sequences...  [0.295s]
Masking sequences...  [1.457s]
Writing sequences...  [0.07s]
Hashing sequences...  [0.022s]
Loading sequences...  [0s]
Writing trailer...  [0s]
Closing the input file...  [0s]
Closing the database file...  [0.001s]

Database sequences  142246
  Database letters  78752603
     Database hash  63a90e7790c6476a9be89aaadd1b422d
        Total time  1.851000s
diamond v2.1.6.160 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021

In [59]:
matches("train_cafa5_db", "/kaggle/working/test.fasta", "cafa5_matches.fasta", 16)
matches("train_cafa6_db", "/kaggle/working/test.fasta", "cafa6_matches.fasta", 16)
matches("train_uniprot_db", "/kaggle/working/test.fasta", "uniprot_matches.fasta", 16)

In [60]:
def create_go_map(terms_file, csv_file):
    go_map = defaultdict(list)
    df = pd.DataFrame()
    if csv_file:
        df = pd.read_csv(terms_file, sep = ",")
    else:
        df = pd.read_csv(terms_file, sep = "\t")
    
    for entry, term in zip(df['EntryID'], df['term']):
        go_map[entry].append(term)

    return go_map


In [62]:
go_map_cafa6 = create_go_map("/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv", False)
go_map_cafa5 = create_go_map("/kaggle/input/cafa5-additional-data/train_terms.tsv", False)
go_map_uniprot = create_go_map("/kaggle/input/final-uniprot/uni_term.tsv", False)

In [63]:
#blast knn
from collections import defaultdict
import pandas as pd

def compute_knn(hits_file, go_map, evalue_thresh=0.001):
    # đọc hits
    hits_df = pd.read_csv(hits_file, sep="\t", header=None,
                          names=['query_id','subject_id','pid','aln_len','mismatch','gap',
                                 'qstart','qend','sstart','send','evalue','bitscore'])
    
    results = defaultdict(dict)
    
    for q, group in hits_df.groupby('query_id'):
        # lọc e-value < threshold
        group = group[group['evalue'] < evalue_thresh]
        if group.empty:
            continue
        
        Bsum = group['bitscore'].sum()
        go_score = defaultdict(float)
        
        for _, row in group.iterrows():
            subject = row['subject_id']
            bits = row['bitscore']
            for go in go_map.get(subject, []):
                go_score[go] += bits
        
        # normalize theo tổng bitscore
        for go in go_score:
            go_score[go] /= Bsum
        
        results[q] = dict(go_score)
    
    return results


In [64]:
results_cafa_6 = compute_knn("/kaggle/working/cafa6_matches.fasta", go_map_cafa6)
results_cafa_5 = compute_knn("/kaggle/working/cafa5_matches.fasta", go_map_cafa5)
results_uniprot = compute_knn("/kaggle/working/uniprot_matches.fasta", go_map_uniprot)

In [65]:
def merge_many_results(*results_dicts, outfile=None):
    """
    Gộp nhiều dict kiểu:
        dict[query_id] -> dict[GO_term] -> score
    
    Nếu cùng query_id và GO_term -> lấy score lớn nhất.
    
    Parameters:
        *results_dicts: danh sách các dict đầu vào
        outfile: nếu muốn xuất TSV -> truyền tên file
    
    Returns:
        final_results: dict đã gộp
    """
    
    final_results = defaultdict(dict)
    
    # duyệt qua từng dict trong danh sách
    for res in results_dicts:
        for q, go_dict in res.items():
            for go, score in go_dict.items():
                if go in final_results[q]:
                    final_results[q][go] = max(final_results[q][go], score)
                else:
                    final_results[q][go] = score
    
    # nếu muốn xuất file
    if outfile is not None:
        with open(outfile, "w") as f:
            f.write("query_id\tGO_term\tscore\n")
            for q, go_dict in final_results.items():
                for go, score in go_dict.items():
                    f.write(f"{q}\t{go}\t{score:.6f}\n")
    
    return final_results


In [66]:
final = merge_many_results(results_cafa_6, results_cafa_5, results_uniprot, outfile = "submission.tsv")